In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections

In [26]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [245]:
possible_plans = [
        ['orange','black','green'],
        ['yellow','black','blue'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [328]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
"""
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
    def __str__(self):
        return str(self.state)


class State():
    X = [0,1,2]
    def __init__(self, lines, heights = [0,0,0], x = 0, y = 0, time = 0):
        self.lines = lines
        self.heights = heights
        self.x = x
        self.y = y
        self.time = time
    
       
    def get_next_allowed_cube(self):
        a = 4
        for l in self.lines:
            for c in l:
                if c > 0:
                    a = min(c, a)
        # return 4 if whole plan picked
        return a
    
    def get_next_line(self):
        for x, l in zip(self.X, self.lines):
            any_cube = False
            for c in l:
                if c != -1:
                    any_cube = True
            if any_cube:
                return x,l
            
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.lines:
            s += str(l) + '\n'
        s += "x=%d, y=%d, time=%f\n"%(self.x, self.y, self.time)
        s += "height = " + str(self.heights) + '\n'
        s += '-----------'
        return s
    
    def is_final(self):
        # if all is -1
        return sum(sum(self.lines)) == -9
    
class SuctionRobotStrategyOptimizer():
    defaults = {
        'plan': [0,1,2],
        'allowed_sides': [
            [False, True, True, True],
            [True, False, True, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, True, False]
        ],
        'heaps': [0,1,2],
        'pick_time': 3,
        'move_time': 1,
        'heap': {'orange':np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ]),
                 'green':np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ])
            }
    }
    def __init__(self, **kvargs):
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
        self.manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
        
    def rotate_heap(self, heap, side):
        return np.rot90(heap, side)
    
    def get_heap(self, number = 0, side = 0):
        heap = None
        if number in [0,1,2]:
            heap = self.params['heap']['orange']
        if number in [3,4,5]:
            heap = self.params['heap']['green']
        return self.rotate_heap(heap,side)
    
    def set_plan(self, plan):
        if isinstance(plan[0], str):
            self.params['plan'] = [colormap[c] for c in plan]
        else:
            self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num in [0,1,2]:
            return self.params['plan'][num]
        return self.params['plan']
    
    def reverse_plan(self):
        self.params['plan'] = self.params['plan'][::-1]
        
    def get_move_time(self, dx, dy):
        return max(abs(dx),abs(dy))*self.params['move_time']
    
    def get_lines_by_plan_and_heap(self, heap):
        lines = copy.copy(heap)
        for i, j in iterer(3,3):
            if heap[i][j] in range(5):
                if heap[i][j] in self.get_plan():
                    lines[i][j] = self.get_plan().index(heap[i][j]) + 1
                else:
                    lines[i][j] = 0
        return lines
    
    def try_to_pick(self, state, y, l, c, x, man, mans):
        picked = [-1,-1,-1]
        for i in range(3): # run over mans
            if mans[i] == 1: # if need to pick by i-th man
                if i + y in [0,1,2] and l[i+y] >= 0: # check if we pick existing cube
                    # print(i,y)
                    if (i == man and (c == l[i+y] or c == 1)) or (i != man and l[i+y] == 0) : # if we can pick this cube according to plan 
                        picked[i] = l[i+y] # add to picked (why?)
                        state.heights[i] += 1 # increase height of i-th man
                        state.lines[x][i+y] = -1 # remove cube from heap
                        # print(i, y, i+y)
                    else:
                        return None
                else:
                    return None
        # if still not return, add time and return new state
        state.time += self.params['pick_time']
        state.time += self.get_move_time(x-state.x, y-state.y)
        state.x = x
        state.y = y
        return state
    
    def get_new_states(self, state, man):
        if not state.is_final():
            c = state.get_next_allowed_cube()
            x, l = state.get_next_line()
            for mans, y in iterer(self.manipulator_masks, range(-2,3)):
                new_state = self.try_to_pick(copy.deepcopy(state), y, l, c, x, man, mans)
                if new_state != None:
                    # print(y, mans)
                    yield new_state   
    
    def reduce_final_states(self, final_states):
        # we can choose only same heights with best times
        heights = {}
        for sn in final_states:
            s = sn.state
            key = str(s.heights)
            if key in heights:
                if heights[key].state.time > s.time:
                    heights[key] = sn
            else:
                heights[key] = sn
        return list(heights.values())
    
    def build_solution_tree(self, heap_num, side, man):
        h = self.get_heap(heap_num, side)
        initial_state = State(opt.get_lines_by_plan_and_heap(h))
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        N = 0
        final_states = []
        while len(this_wave) > 0:
            for node in this_wave:
                if node.state.is_final():
                    final_states.append(node)
                node.nexts = [STNode(x,node) for x in self.get_new_states(node.state, man)]
                next_wave.extend(node.nexts)
            if len(next_wave) > 0:
                N = len(next_wave)
            this_wave = next_wave
            next_wave = []
        return root, self.reduce_final_states(final_states)
    
    def count_all_variants(self, progress_bar=False):
        team_colors, sides, mans = 2, 4, 3
        variants = []
        progress = 0
        for team_color, side, man in iterer(team_colors, sides, mans):
            _, final_states = self.build_solution_tree(team_color*3, side, man)
            for fs in final_states:
                variants.append([team_color, side, man, fs])
            progress += 1
            if progress_bar:
                print("%d%s" % (progress / (team_colors*sides*mans)*100, '%')) 
        return variants
    
    def count_all_picking_plans(self, progress_bar=False):
        variants = self.count_all_variants()
        
        heights = {}
        for v in variants:
            key = str(v[-1].state.heights)
            if key in heights:
                heights[key].append(v)
            else:
                heights[key]= [v]
                    
        path_variants = []
        i = 0
        total = len(variants)**2
        for v1,v2 in iterer(variants,variants):
            height_left = [5-h1-h2 for h1,h2 in zip(v1[-1].state.heights, v2[-1].state.heights)]
            key = str(height_left)
            if key in heights:
                for v3 in heights[key]:
                    path_variants.append([v1,v2,v3])
            i+=1
            if progress_bar:
                print("%d%s" % (i / total *100, '%')) 
        return path_variants
    
    def count_best_picking_plan(self):
        path_variants = self.count_all_picking_plans()
        min_v = path_variants[0]
        min_t = sum([v[-1].state.time for v in min_v])
        for pv in path_variants:
            pv_t = sum([v[-1].state.time for v in pv])
            if pv_t < min_t:
                min_v = pv
                min_t = pv_t
        return min_v

In [329]:
opt = SuctionRobotStrategyOptimizer()

In [332]:
for p in possible_plans:
    opt.set_plan(p)
    best_pv = opt.count_best_picking_plan()
    for v in best_pv:
        print(v[:3],v[-1])
    total_time = sum([v[-1].state.time for v in best_pv])
    print(total_time)

[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=16.000000
height = [1, 3, 1]
-----------
[0, 1, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 1, 1]
-----------
[0, 3, 2] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=12.000000
height = [1, 1, 3]
-----------
40
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=16.000000
height = [4, 1, 0]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=0, time=11.000000
height = [1, 3, 1]
-----------
[0, 0, 2] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=16.000000
height = [0, 1, 4]
-----------
43
[0, 1, 2] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=12.000000
height = [1, 1, 3]
-----------
[0, 2, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=16.000000
height = [1, 3, 1]
-----------
[0, 3, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 1, 1]
---

In [326]:
for v in best_pv:
    print(v[:3],v[-1])

[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [1, 3, 1]
-----------
[0, 1, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=9.000000
height = [3, 1, 1]
-----------
[0, 3, 2] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=9.000000
height = [1, 1, 3]
-----------


In [300]:
for p in possible_plans:
    opt.set_plan(p)
    print(len(opt.count_time_matrix()))

120
132
126
132
120
120
120
96
102
120


In [295]:
for v in variants:
    print(v[:3],v[-1])

[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=9.000000
height = [3, 1, 1]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=0, time=9.000000
height = [3, 2, 0]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=11.000000
height = [3, 0, 2]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=11.000000
height = [4, 0, 1]
-----------
[0, 0, 0] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=0, time=11.000000
height = [4, 1, 0]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000
height = [1, 3, 1]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000
height = [2, 3, 0]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=8.000000
height = [0, 3, 2]
-----------
[0, 0, 1] ---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=-1, time=10.000000
height = [0, 4, 1]
-----------
[

In [296]:
len(variants)

120

In [250]:
%%time
for str_plan in possible_plans:
    opt.set_plan(str_plan)
    root, final_states = opt.build_solution_tree(0,0,0)
    print(len(final_states))

5
6
6
5
3
6
6
5
0
6
CPU times: user 448 ms, sys: 12 ms, total: 460 ms
Wall time: 422 ms


In [241]:
opt.set_plan([colormap[c] for c in str_plan])
root, fs = opt.build_solution_tree(0,0,0)
for sn in fs:
    print(sn.state)
    
print('--------------------------------------_')
sn = fs[0]
while sn != None:
    print(sn.state)
    sn = sn.upper

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=11.000000
height = [3, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [4, 0, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [3, 2, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=9.000000
height = [4, 1, 0]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=10.000000
height = [5, 0, 0]
-----------
--------------------------------------_
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=12.000000
height = [3, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  2 -1]
x=1, y=2, time=10.000000
height = [2, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1  1]
[-1  2 -1]
x=1, y=1, time=8.000000
height = [1, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1  3  1]
[-1  2 -1]
x=1, y=-

In [218]:
for sn in final_states:
    print(sn.state)

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=6.000000
height = [3, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=9.000000
height = [3, 0, 2]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=8.000000
height = [3, 2, 0]
-----------


In [219]:
sn = final_states[0]
while sn != None:
    print(sn.state)
    sn = sn.upper

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=6.000000
height = [3, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  3 -1]
x=1, y=0, time=4.000000
height = [2, 1, 1]
-----------
---STATE---
[-1 -1 -1]
[2 0 0]
[-1  3 -1]
x=0, y=1, time=2.000000
height = [1, 0, 0]
-----------
---STATE---
[-1  1 -1]
[2 0 0]
[-1  3 -1]
x=0, y=0, time=0.000000
height = [0, 0, 0]
-----------
